# Section 4: The Art of Phase Control and Interference

**Objective:** Dive deeper into how phase is manipulated in quantum circuits and how quantum interference leads to computational power.

---

### Key Concepts:

* **Phase as Quantum Control:**
    * **Intuition:** In classical telecommunications, we often encode information by modulating the phase of a carrier wave. Similarly, in quantum computing, quantum operations can precisely control the *phase* of a qubit's quantum state.
    * This control over phase is subtle but incredibly powerful, as it's the key ingredient for creating quantum interference effects.

* **Quantum Interference:**
    * **Intuition:** Think of waves in water. When two waves meet, they can either reinforce each other (constructive interference) or cancel each other out (destructive interference).
    * In quantum computing, this principle applies to probability amplitudes. Quantum algorithms are designed such that:
        * **Undesirable outcomes** (wrong answers) have their probability amplitudes interfere destructively and "cancel out."
        * **Desirable outcomes** (correct answers) have their probability amplitudes interfere constructively and "reinforce" each other.
    * This intelligent manipulation of interference is where the exponential speedup of quantum algorithms comes from. It's how a quantum computer can explore many possibilities simultaneously and arrive at the correct solution with high probability, seemingly faster than a classical computer. The QFT, for example, heavily relies on precise phase rotations and subsequent interference to extract frequency information.

---

### Implementation: Demonstrating Controlled Phase and Interference

Let's observe how a controlled-phase gate manipulates the phase of a qubit and how this can lead to different measurement probabilities after subsequent operations, demonstrating interference effects.

We will use a **Controlled-Phase (CP) gate**, also known as `CU1` or `CRZ` in some contexts, or constructed from `CX` and `RZ` gates. The `CP(theta)` gate applies a phase `e^(i*theta)` to the target qubit *only if* the control qubit is in the $|1\rangle$ state. If the control is $|0\rangle$, nothing happens to the target qubit's phase.

We'll set up a 2-qubit circuit:
1.  Put both qubits into superposition using Hadamard gates.
2.  Apply a controlled-phase gate.
3.  Apply a final Hadamard to the target qubit.
4.  Measure both qubits.

In [ ]:
%matplotlib inline
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np # For defining phase angles

# Create a 2-qubit circuit with 2 classical bits
qc_phase_control = QuantumCircuit(2, 2)

# --- Setup for Interference Demonstration ---
# Apply Hadamard to both qubits to put them into superposition
qc_phase_control.h(0)
qc_phase_control.h(1)
qc_phase_control.barrier() # Visual separator

# --- Apply a Controlled-Phase Gate ---
# The 'theta' value determines the phase shift. Try changing this!
# np.pi / 2 (90 degrees) or np.pi (180 degrees) are common to see effects.
# Example: theta = np.pi / 2 (CP(pi/2) gate)
theta_angle = np.pi / 2  # Experiment with np.pi, np.pi/4, 0, etc.
print(f"Applying a Controlled-Phase gate with theta = {theta_angle:.2f} radians ({np.degrees(theta_angle):.0f} degrees)")

# Apply the controlled-phase gate (control_qubit=0, target_qubit=1)
qc_phase_control.cp(theta_angle, 0, 1)

# An alternative way to implement a controlled-phase effect using standard gates:
# If theta_angle is not zero, uncomment these lines and comment qc_phase_control.cp above
# qc_phase_control.p(theta_angle / 2, 0) # Phase on control qubit
# qc_phase_control.p(theta_angle / 2, 1) # Phase on target qubit
# qc_phase_control.cx(0, 1)              # CNOT
# qc_phase_control.p(-theta_angle / 2, 1) # Inverse phase on target
# qc_phase_control.cx(0, 1)              # CNOT

qc_phase_control.barrier() # Visual separator

# --- Final Hadamard for Interference ---
# Apply Hadamard to the target qubit (Qubit 1) to make phase differences visible in measurement probabilities
qc_phase_control.h(1)
qc_phase_control.barrier() # Visual separator

# --- Measure both qubits ---
# As discussed, Qiskit's bitstrings are c1c0, where c0 is from Qubit 0, c1 from Qubit 1
qc_phase_control.measure([0, 1], [0, 1])

# Draw the circuit
print("Controlled-Phase & Interference Circuit Diagram:")
qc_phase_control.draw(output="mpl")
plt.show()

# Select the Aer simulator backend
simulator = AerSimulator()

# Execute the circuit
job = simulator.run(qc_phase_control, shots=1024)
result = job.result()

# Get the measurement counts
counts = result.get_counts(qc_phase_control)

print("\nMeasurement Counts:")
print(counts)

# Plot the results
plot_histogram(counts).set_size_inches(7, 5)
plt.title(f"Controlled-Phase (theta={theta_angle:.2f}) Interference Results")
plt.show()

print("\nExperiment with different 'theta_angle' values (e.g., 0, np.pi/2, np.pi) and observe how the probabilities of the '01' and '11' outcomes change. This demonstrates how subtle phase shifts, when followed by a Hadamard, can lead to dramatic changes in measurement probabilities due to quantum interference. This is the heart of how many quantum algorithms work!")

### Visualizing Phase (Optional)

While we won't implement a full visualization here, you can explore the phase of qubits more directly using Qiskit's `StatevectorSimulator` and visualization tools like `plot_bloch_multivector` or `plot_state_qsphere` after intermediate steps in a circuit (before measurement). These tools help visualize how the Bloch sphere representation of a qubit changes, including its phase, which is crucial for understanding interference.

For instance, you could run a circuit up to the point of the `cp` gate, get the statevector, and then visualize it to see the phase changes. However, for a two-qubit system, `plot_bloch_multivector` can become complex to interpret quickly, and `plot_state_qsphere` is often more suitable for multi-qubit phase visualization. These are topics for deeper dives!